In [179]:
# Packages
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import pandas as pd

# The Pages I'm Using
urls = ['passing.htm', 'rushing.htm', 'receiving.htm']
base_url = 'https://www.pro-football-reference.com/years/2019/'

# To iterate through each page
for addon in urls:
    searching_url = base_url + addon
    url_client = urlopen(searching_url)
    html_url = url_client.read()
    url_client.close()
    soup_url = soup(html_url, 'html.parser')

    # To find all stats listed
    container_1 = soup_url.body.div.table.findAll('th', {'scope':"col"}) # holds all stat types
    num_players = range(len(soup_url.body.div.table.findAll('td',{'data-stat' : "player"})))
    num_stats = len(container_1)
    stats = []
    count = 0

    for _ in container_1: 
        searching = container_1[count] # the place on the chart I'm looking at
        value = searching['data-stat'] # the value there 
        stats.append(value)            # adds the found value to a list of all stats
        count += 1

    all_stats = {}
    for _ in stats:
        all_stats[_] = [] # for each stat name 

    # Creates a dataframe for each page
    if addon == urls[0]:
        passing_df = pd.DataFrame(all_stats)
    if addon == urls[1]:
        rushing_df = pd.DataFrame(all_stats)
    if addon == urls[2]:
        receiving_df = pd.DataFrame(all_stats)

    count = 0
    all_stats = soup_url.body.div.table.findAll('td') # returns a list of every stat across all players on the page

    # Pulls 1 rows worth of stats adds it to the appropriate dataframe and moves to the next row
    for _ in num_players:
        stat_dict = {}

        for _ in range(num_stats - 1): # pulls the value for each stat across a column and restarts once it reaches the end of a column
            searching = all_stats[count] 
            stat_dict[searching['data-stat']] = all_stats[count].text # dict entry as {stat name : stat value}
            count += 1
        
        # adds 1 row (1 player) to the appropriate df the key determines where the value gets placed
        if addon == urls[0]:
            passing_df = passing_df.append(stat_dict, ignore_index = True)
        if addon == urls[1]:
            rushing_df = rushing_df.append(stat_dict, ignore_index = True)
        if addon == urls[2]:
            receiving_df = receiving_df.append(stat_dict, ignore_index = True)

       # from here the algorithm(?) repeats for each player on a page, then through each page      
        
# Merging all dataframes into one, dropping stats I dont want, in filling in NaNs, and indexing by name
passing_and_rushing = rushing_df.merge(passing_df, how = 'outer', on = None)
df = passing_and_rushing.merge(receiving_df, how = 'outer', on = None)
df = df.drop(['pass_td_perc', 'qbr','pass_adj_net_yds_per_att','pass_sacked_yds','rec_yds_per_tgt','rec_per_g','rec_long','pass_sacked_perc','pass_net_yds_per_att','pass_sacked','pass_rating','pass_yds_per_g','pass_yds_per_cmp','pass_adj_yds_per_att','pass_yds_per_att','pass_long','pass_first_down','pass_int_perc','pass_cmp_perc','qb_rec','rush_yds_per_g','rush_yds_per_att','rush_long','rec_first_down','rec_yds_per_g','rec_yds_per_rec','comebacks','gs','gwd','rush_first_down','ranker',], axis = 1)
df = df.fillna(value = 0)
df = df.set_index('player')

# Converts all numeric stats to INT except catch %
used_stats = [] 
for _ in df.columns: # did it this way because .columns returns a series and it was giving me issues
    used_stats.append(_)   
    
int_stats = used_stats[3:15] + used_stats[16:18]  #ADJUST ME IF YOU CHANGE THE STATS YOU ARE USING

for _ in int_stats:
    df[_]= df[_].astype(int)

# Stats I'm creating
df['TDs'] = df['rush_td'] + df['rec_td']
df['Points'] = 0.1*df['rush_yds'] + 0.1*df['rec_yds'] + 0.04*df['pass_yds'] + 6*df['rush_td'] + 6*df['rec_td'] + 4*df['pass_td'] - 2*df['pass_int'] - 2*df['fumbles']
df['PPG'] = df['Points'] / df['g']
df = df.sort_values('Points', ascending = False)

# Splits dfs by roster spots
WRs_df = df.loc[df['pos'].isin(['WR'])]
RBs_df = df.loc[df['pos'].isin(['RB'])]
Flex_df = df.loc[df['pos'].isin(['WR', 'RB', 'TE'])]
QBs = df.loc[df['pos'].isin(['QB'])]

In [182]:
RBs_df.sort_values('PPG', ascending = False).head(12)

,team,age,pos,g,rush_att,rush_yds,rush_td,fumbles,pass_cmp,pass_att,...,pass_td,pass_int,targets,rec,catch_pct,rec_yds,rec_td,TDs,Points,PPG
player,,,,,,,,,,,,,,,,,,,,,
Christian McCaffrey*+,CAR,23,RB,16,287,1387,15,1,0,2,...,0,0,142,116,81.7%,1005,4,19,351.2,21.950000
Derrick Henry*,TEN,25,RB,15,303,1540,16,5,0,0,...,0,0,24,18,75.0%,206,2,18,272.6,18.173333
Dalvin Cook*,MIN,24,RB,14,250,1135,13,4,0,0,...,0,0,63,53,84.1%,519,0,13,235.4,16.814286
Aaron Jones,GNB,25,RB,16,236,1084,16,3,0,0,...,0,0,68,49,72.1%,474,3,19,263.8,16.487500
Ezekiel Elliott*,DAL,24,RB,16,301,1357,12,3,0,0,...,0,0,71,54,76.1%,420,2,14,255.7,15.981250
Saquon Barkley,NYG,22,RB,13,217,1003,6,1,0,0,...,0,0,73,52,71.2%,438,2,8,190.1,14.623077
Mark Ingram*,BAL,30,RB,15,202,1018,10,2,0,0,...,0,0,29,26,89.7%,247,5,15,212.5,14.166667
Nick Chubb*,CLE,24,RB,16,298,1494,8,3,0,0,...,0,0,49,36,73.5%,278,0,8,219.2,13.700000
Austin Ekeler,LAC,24,RB,16,132,557,3,3,0,0,...,0,0,108,92,85.2%,993,8,11,215.0,13.437500
